In [11]:
!pip install osmnx networkx shapely ipyleaflet ipywidgets

In [ ]:
import osmnx as ox
import networkx as nx
import ipyleaflet as ipl
import pickle
import shapely

from shapely.geometry import Polygon
from ipyleaflet import Map, Marker, FullScreenControl, AwesomeIcon
from ipywidgets import interact
from time import sleep

In [12]:
# Setting up graph for CMRIT
## Using boundbox

# north, south, east, west = 12.96768338736144,12.965518842636625,77.71393642911939,77.71046157295785
# # cmrit lat long bounds 0
# # north (float) – northern latitude of bounding box: 12.96768338736144
# # south (float) – southern latitude of bounding box: 12.965518842636625
# # east (float) – eastern longitude of bounding box: 77.71393642911939
# # west (float) – western longitude of bounding box: 77.71046157295785
# network_type = 'all_private' # "all_private", "all", "bike", "drive", "drive_service", "walk"

# G = ox.graph_from_bbox(
#       north, south, east, west, 
#       network_type=network_type, 
#       simplify=False, 
#       retain_all=False, 
#       truncate_by_edge=True, 
#       clean_periphery=True, 
#       custom_filter=None)

# Setting up the graph using the pickle file
filename = "../output/geocode_tree.pkl"

with open(filename, "rb") as f:
    G = pickle.load(f)


# Plot the graph
fig, ax = ox.plot_graph(G)

ModuleNotFoundError: No module named 'shapely.io'

In [ ]:
sorted(G.nodes)

In [ ]:
# Finding the shortest path with the given start and end lat-lng
optimizer = 'length'

start_latlng = (12.96751, 77.71388) # cmrit-entrance
end_latlng = (12.96606,77.71146) # ganesha-turn

orig_node = ox.distance.nearest_nodes(G, start_latlng[1], start_latlng[0])
dest_node = ox.distance.nearest_nodes(G, end_latlng[1], end_latlng[0])

shortest_route = nx.shortest_path(G, orig_node, dest_node, weight=optimizer)

shortest_route_map = ox.plot_route_folium(G, shortest_route)
shortest_route_map

In [ ]:
# To find the coordinates in the shortest path
route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_route]
route_coords

In [ ]:
# Plotting the interactive map with the center at (12.96682, 77.71231) => This center is for the map plotting not related with the graph
center = (12.96682, 77.71231)
m1 = ipl.Map(center=center, basemap=ipl.basemaps.CartoDB.Positron, zoom=18)
m1.layout.height = '800px'

In [ ]:
# Drawing the path in the interactive map
path = route_coords
lines = ipl.Polyline(
    locations=path,
    color="tomato",
    fill=False
)
m1.add_layer(lines)

In [ ]:
# Adding markers in the interactive map

icon1 = AwesomeIcon(
    name='fa-truck',
    marker_color='green',
    icon_color='black',
    spin=False
)

marker = ipl.Marker(icon=icon1, location=start_latlng)
m1.add_layer(marker)
m1

In [ ]:
# Animation
# Moving the marker according to the path
step = 0.1 # Speed can be varied here... higher the value lower the speed
start_location = marker.location

for point in path:
    marker.location = point
    sleep(step)


In [ ]:
"""
NOTE:: Don't interact with the map as the kernel will get busy and it will crash
The animation will be shown after the map is drawn
"""